In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
from skimage.feature import greycomatrix, greycoprops

# Entropia

In [ ]:
imagen_referencia = np.asarray([[123,123,125], [145,154,165], [145,155,165]])
imagen_capturada = np.asarray([[123,124,125], [146,156,164], [145,154,162]])

In [ ]:
def entropia(image):
    label, count  = np.unique(image, return_counts= True)
    probList = []
    for i in count:
        probList.append(-np.log2(i/image.size) * (i/image.size)) 
    return sum(probList)

In [ ]:
def d_cuadrado(imagen_referencia, imagen_capturada):
    return np.sum(pow(imagen_referencia - imagen_capturada, 2)/imagen_capturada.size)

In [ ]:
def signalNoiseRatio(imagen_referencia, imagen_capturada):
    return 20*np.log10( imagen_referencia.mean() / np.sqrt(d_cuadrado(imagen_referencia, imagen_capturada)))

In [ ]:
def PsignalNoiseRatio(imagen_referencia, imagen_capturada, maxrange = 255):
    return 20*np.log10( maxrange / np.sqrt(d_cuadrado(imagen_referencia, imagen_capturada)))

In [ ]:
entropia_referencia =  entropia(imagen_referencia)
entropia_capturada =  entropia(imagen_capturada)
print('entropia imagen de referencia ', entropia_referencia)
print('entropia imagen capturada ', entropia_capturada)
print('aumento de entropia ', entropia_capturada - entropia_referencia)

entropia imagen de referencia  2.503258334775646
entropia imagen capturada  3.169925001442312
aumento de entropia  0.6666666666666661


In [ ]:
print('d_cuadrado :', d_cuadrado(imagen_referencia, imagen_capturada))
print('R_media: ', imagen_referencia.mean())
print('SNR :', signalNoiseRatio(imagen_referencia, imagen_capturada))
print('PSNR :', PsignalNoiseRatio(imagen_referencia, imagen_capturada))

d_cuadrado : 1.8888888888888888
R_media:  144.44444444444446
SNR : 40.43195273796074
PSNR : 45.36873948928961


# Imagen ecualizada

In [ ]:
imagen = np.asarray([[5,3,2,7,2], [1,5,7,0,2], [5,6,4,2,0], [1,3,2,7,3], [3,0,7,3,4]])

In [ ]:
def imagen_ecualizada(imagen):
    i, h_i  = np.unique(imagen, return_counts= True)
    H_i = np.cumsum(h_i)
    base = (len(i) - 1)/imagen.size
    s_i = np.round(H_i*base)
    
    df =  pd.DataFrame([i, h_i, H_i, s_i], dtype=int).T
    df.columns=['i', 'h_i', 'H_i', 's_i']
    
    imagen_ecualizada = s_i[imagen]
    
    return i, h_i, H_i, s_i, df, imagen_ecualizada

In [ ]:
i, h_i, H_i, s_i, df, imagen_ecualizada = imagen_ecualizada(imagen)

In [ ]:
print('tablas ')
print(df)
print('imagen_ecualizada: ')
print(imagen_ecualizada)

tablas 
   i  h_i  H_i  s_i
0  0    3    3    1
1  1    2    5    1
2  2    5   10    3
3  3    5   15    4
4  4    2   17    5
5  5    3   20    6
6  6    1   21    6
7  7    4   25    7
imagen_ecualizada: 
[[6. 4. 3. 7. 3.]
 [1. 6. 7. 1. 3.]
 [6. 6. 5. 3. 1.]
 [1. 4. 3. 7. 4.]
 [4. 1. 7. 4. 5.]]


# Matriz de confusion y IoU

In [ ]:
verdad = np.asarray(    [['R', 'R', 'G', 'G',], ['R', 'R', 'G', 'G',], ['B', 'B', 'B', 'G',], ['B', 'B', 'B', 'G',]])
prediccion = np.asarray([['R', 'R', 'G', 'G',], ['B', 'R', 'G', 'G',], ['B', 'B', 'G', 'G',], ['B', 'B', 'B', 'G',]])

In [ ]:
elementos = np.unique(verdad)
elementos

array(['B', 'G', 'R'], dtype='<U1')

In [ ]:
matrix = confusion_matrix(verdad.flatten(), prediccion.flatten(), labels=['B', 'G', 'R'])
matrix

array([[5, 1, 0],
       [0, 6, 0],
       [1, 0, 3]])

In [ ]:
valores = []
for elemento in elementos:
    a = verdad.flatten() == elemento
    b = prediccion.flatten() == elemento
    union = 0
    interseccion = 0
    for i in range(len(a)):
        if a[i] == b[i]:
            if a[i] == True:
                interseccion += 1
                union += 1
        else:
            union += 1
    print('Elemento ', elemento, ':', interseccion, '/', union)
    valores.append(interseccion/union)
valores    

Elemento  B : 5 / 7
Elemento  G : 6 / 7
Elemento  R : 3 / 4


[0.7142857142857143, 0.8571428571428571, 0.75]

In [ ]:
iou = (1/len(matrix))*sum(valores)
iou

0.7738095238095237

# Magnitud y modulo

In [ ]:
imagen = np.asarray([[1,2,6,4], [3,7,5,3], [9,5,8,6]])

In [ ]:
dy, dx = np.gradient(imagen)
dx, dy

(array([[ 1. ,  2.5,  1. , -2. ],
        [ 4. ,  1. , -2. , -2. ],
        [-4. , -0.5,  0.5, -2. ]]), array([[ 2. ,  5. , -1. , -1. ],
        [ 4. ,  1.5,  1. ,  1. ],
        [ 6. , -2. ,  3. ,  3. ]]))

In [ ]:
G = np.sqrt(pow(dx, 2) + pow(dy, 2))
G

array([[2.23606798, 5.59016994, 1.41421356, 2.23606798],
       [5.65685425, 1.80277564, 2.23606798, 2.23606798],
       [7.21110255, 2.06155281, 3.04138127, 3.60555128]])

In [ ]:
np.degrees(np.arctan(dy/dx))

array([[ 63.43494882,  63.43494882, -45.        ,  26.56505118],
       [ 45.        ,  56.30993247, -26.56505118, -26.56505118],
       [-56.30993247,  75.96375653,  80.53767779, -56.30993247]])

# Matrices de Haralick

In [ ]:
imagen = np.asarray([[1,2,1,3], [0,1,1,2], [1,2,3,3], [2,2,3,3]])

In [ ]:
# en clase se ha visto 90 grados hacia arriba, pero python lo interpreta hacia abajo, por eso -pi/2
matriz_co_ocurrencia = greycomatrix(imagen, distances=[1], angles=[0, np.pi/4, -np.pi/2, 3*np.pi/4], levels=4)

In [ ]:
print('Matrices de Haralick G')
print()
print('0 grados')
print(matriz_co_ocurrencia[:, :, 0, 0])
print()
print('45 grados')
print(matriz_co_ocurrencia[:, :, 0, 1])
print()
print('90 grados')
print(matriz_co_ocurrencia[:, :, 0, 2])
print()
print('135 grados')
print(matriz_co_ocurrencia[:, :, 0, 3])

Matrices de Haralick G

0 grados
[[0 1 0 0]
 [0 1 3 1]
 [0 1 1 2]
 [0 0 0 2]]

45 grados
[[0 0 1 0]
 [0 1 2 2]
 [0 1 0 1]
 [0 0 0 1]]

90 grados
[[0 1 0 0]
 [1 1 1 0]
 [0 2 1 1]
 [0 1 1 2]]

135 grados
[[0 0 0 0]
 [0 2 1 0]
 [1 0 1 1]
 [0 1 1 1]]


In [ ]:
matriz_co_ocurrencia = greycomatrix(imagen, distances=[1], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4], levels=4, normed=True)
print('Matrices de Haralick G NORMALIZADAS')
print()
print('0 grados')
print(matriz_co_ocurrencia[:, :, 0, 0])
print()
print('45 grados')
print(matriz_co_ocurrencia[:, :, 0, 1])
print()
print('90 grados')
print(matriz_co_ocurrencia[:, :, 0, 2])
print()
print('135 grados')
print(matriz_co_ocurrencia[:, :, 0, 3])

Matrices de Haralick G NORMALIZADAS

0 grados
[[0.         0.08333333 0.         0.        ]
 [0.         0.08333333 0.25       0.08333333]
 [0.         0.08333333 0.08333333 0.16666667]
 [0.         0.         0.         0.16666667]]

45 grados
[[0.         0.         0.11111111 0.        ]
 [0.         0.11111111 0.22222222 0.22222222]
 [0.         0.11111111 0.         0.11111111]
 [0.         0.         0.         0.11111111]]

90 grados
[[0.         0.08333333 0.         0.        ]
 [0.08333333 0.08333333 0.16666667 0.08333333]
 [0.         0.08333333 0.08333333 0.08333333]
 [0.         0.         0.08333333 0.16666667]]

135 grados
[[0.         0.         0.         0.        ]
 [0.         0.22222222 0.11111111 0.        ]
 [0.11111111 0.         0.11111111 0.11111111]
 [0.         0.11111111 0.11111111 0.11111111]]


In [ ]:
matriz_contrast = greycoprops(matriz_co_ocurrencia, prop='contrast')
matriz_contrast

array([[0.91666667, 1.77777778, 0.91666667, 1.22222222]])

In [ ]:
matriz_homogeneity = greycoprops(matriz_co_ocurrencia, prop='homogeneity')
matriz_homogeneity

array([[0.64166667, 0.51111111, 0.64166667, 0.65555556]])

In [ ]:
matriz_energy = greycoprops(matriz_co_ocurrencia, prop='energy')
matriz_energy

array([[0.39086798, 0.40061681, 0.33333333, 0.36851387]])

In [ ]:
matriz_homogeneity = greycoprops(matriz_co_ocurrencia, prop='homogeneity')
matriz_homogeneity

array([[0.64166667, 0.51111111, 0.64166667, 0.65555556]])